# Searching L4A CH<sub>4</sub> Records

This tutorial provides steps for querying the L4A CH<sub>4</sub> collection using the Carbon Mapper platform STAC API.

## Installing Requirements

For tutorial purposes, we will use IPython's pip magic command to install the [PySTAC Client](https://github.com/stac-utils/pystac-client) requirement in the current kernel.

In [ ]:
%pip install pystac-client

## Initalizing the Client

Initialize PySTAC-Client and print any errors that occur.

In [ ]:
from pystac_client import Client
from pystac_client.exceptions import APIError

catalog_url = "https://api.carbonmapper.org/api/v1/stac/"

try:
    catalog = Client.open(catalog_url)
except APIError as e:
    for message in json.loads(str(e))["messages"]:
        print("Error:", message["message"])

    raise Exception(e)

## Searching the L4A-CH<sub>4</sub> Collection

Because the Carbon Mapper platform STAC API supports the Query extension, searches can be performed on properties in addition to those defined by the [STAC specifications](https://github.com/radiantearth/stac-api-spec/tree/release/v1.0.0/item-search#query-parameter-table). We will use a specification query parameter, `datetime`, and one additional field, `cm:emission` to find observations of emissions measuring greater than 3,000 kg/hr between 2023 and 2024. After printing the feature ID and recorded emission, we will print the available properties of those features. The additional properties available can vary feature-to-feature, so the list will not be representative of every feature's available properties.

In [ ]:
import json
from datetime import datetime

collections = catalog.get_collections()
print("L4A collections:\n")

for collection in collections:
    if "l4a" in collection.id:
        print(f"\t{collection.id}")

parameters = {
    "max_items": 10,  # Limit results for tutorial
    "datetime": (
        datetime.strptime("20230101", "%Y%m%d"),  # Inclusive start
        datetime.strptime("20240101", "%Y%m%d"),  # Exclusive end
    ),
    "query": {"cm:emission": {"gt": 3000}},  # Emissions greater than 3000 kg/hr
    "collections": "l4a-ch4",
}
search = catalog.search(**parameters)
print("\nFeatures:\n")

for item in list(search.items_as_dicts()):    
    print(
        f"\tEmission for plume with ID {item['id']}: {item['properties']['cm:emission']:.0f} +/-"
        f" {item['properties']['cm:emission_uncertainty']:.0f} kg/hr"
    )

properties = list()

for item in list(search.items_as_dicts()):
    for property in item['properties'].keys():
        if property not in properties:
            properties.append(property)

print("\nFeature properties:\n")
print("\t{}".format("\n\t".join(sorted(properties))))